## Importing gspread module, pandas & service account. 

### Modules and its use cases.

* Gspread: To read, write & update data into google sheets. 
* pprint: To present the data in a better format. 
* service account: To connect with `google api`. 

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import csv
from datetime import date
import json
import requests


scope = ["https://spreadsheets.google.com/feeds",
        'https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("/Users/satvik/Desktop/Project/creds.json", scope)

google_sheet = gspread.authorize(creds)


### Integrating metabase 

* Generating the token 
* Getting the data from the application card i.e. `Query`
* Printing the data

In [2]:


def get_token():
    res = requests.post('http://metabase.frontrow.co.in/api/session', 
                        headers = {"Content-Type": "application/json"},
                        json =  {"username": "user-name", 
                                "password": 'password'}
                    )
    assert res.ok == True
    print("got the token")
    return res.json()['id']

def get_data(token, card_id):
    url = f'http://metabase.frontrow.co.in/api/card/{card_id}/query/json'
    headers = {
        'Content-Type': 'application/json',
        'X-Metabase-Session': token
    }
    print("getting the data")
    response = requests.post(url, headers=headers)

    df = pd.DataFrame(response.json())
    print("saving")
    return df
    

token = get_token()
front_row_csv = get_data(token, 222)

got the token
getting the data
saving


### Creating CSV 
* Converting Data Frame into CSV

In [3]:
from datetime import datetime

now = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

metabase_revenue_sheet = f"/Users/satvik/Desktop/Project/metabase_revenue_sheet_{now}.csv"
front_row_csv.to_csv(metabase_revenue_sheet, index=False)

google_sheet_id = "1XLTtnFKpf6zVqo-5wwsFSvljZJcooBfQ4cnKEp8lZ9Q"
google_sheet.insert_permission(
    google_sheet_id,
    None,
    perm_type='anyone',
    role='writer'
)

#Writing Data into from csv to Master-worksheet
spread_sheet = google_sheet.open_by_key(google_sheet_id)
spread_sheet.values_update(
    'metabase_revenue_data',
    params={'valueInputOption': 'USER_ENTERED'},
    body={'values': list(csv.reader(open(metabase_revenue_sheet)))}
    )


def get_json_response(url):
    headers = {"Authorization": "Bearer {}".format(access_token)}
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    
    dataframe = pd.DataFrame.from_dict(response.json(), orient="index")
    dataframe = dataframe.transpose()
    dataframe= dataframe.drop('paging',1)
    
    
    campaign_id =[]
    campaign_name =[]
    total_spend =[]
    date =[]
    ctr = []
    clicks = []
    cpm = []
    impressions = []

    for i in range(dataframe.shape[0]):
        date.append(dataframe['data'][i]['date_start'])
        campaign_name.append(dataframe['data'][i]['campaign_name'])
        campaign_id.append(dataframe['data'][i]['campaign_id'])
        total_spend.append(dataframe['data'][i]['spend'])
        ctr.append(dataframe['data'][i]['ctr'])
        clicks.append(dataframe['data'][i]['clicks'])
        cpm.append(dataframe['data'][i]['cpm'])
        impressions.append(dataframe['data'][i]['impressions'])
        
        
        
    

    newdataframe = pd.DataFrame({"date":date, 
                                "campaign_name":campaign_name,
                                "campaign_id":campaign_id,
                                "total_cost":total_spend,
                                "impressions":impressions,
                                "cpm":cpm, 
                                "ctr":ctr, 
                                "clicks":clicks, 
                                

                                })
                                
    newdataframe = newdataframe[newdataframe.campaign_name.str.contains('Record')]

    return newdataframe   

access_token = "token here"
url = f"https://graph.facebook.com/v13.0/act_567744690350558/insights?date_preset=this_month&time_increment=1&limit=1000&level=campaign&fields=campaign_name%2Ccampaign_id%2Cspend%2Cctr%2Ccpm%2Cclicks%2Cimpressions&access_token={access_token}"


spends = get_json_response(url)

spends_csv = f"spends_indepth_{now}.csv"
spends.to_csv(spends_csv,index=False)

spread_sheet.values_update(
    'facebook_raw_data',
    params={'valueInputOption': 'USER_ENTERED'},
    body={'values': list(csv.reader(open(spends_csv)))}
    )

/var/folders/qh/3p8ysdvd7_l6wsdl6hm_tv_m0000gn/T/ipykernel_43880/1711510909.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dataframe= dataframe.drop('paging',1)


{'spreadsheetId': '1XLTtnFKpf6zVqo-5wwsFSvljZJcooBfQ4cnKEp8lZ9Q',
 'updatedRange': 'facebook_raw_data!A1:H376',
 'updatedRows': 376,
 'updatedColumns': 8,
 'updatedCells': 3008}

# Integrating Mixpannel Api
* Importing data from mixpannel for: 
* Course view
* Login Success
* Purchases

In [4]:
def get_mixpannel_data(funnel_id):
    from datetime import date
    from datetime import timedelta
    import requests
    today = date.today()

    url = f"https://mixpanel.com/api/2.0/funnels?project_id=2297403&funnel_id={funnel_id}&from_date=2022-05-01&to_date={today}"

    headers = {
        "Accept": "application/json",
        "Authorization": "Basic api"
    }

    response = requests.get(url, headers=headers)
    response = response.json()
    response = json.dumps(response, indent =4)

    return response



In [5]:
def get_metrics(response):
    json_object = json.loads(response)
    
    dates_list = list(json_object['data'].keys())

    dates_list.sort()
    
    course_view = []
    login_success = []
    course_id =[]

        
    for dates in dates_list:
        course_view.append(json_object['data'][dates]['$overall'][0]['count'])
        login_success.append(json_object['data'][dates]['$overall'][1]['count'])
        course_id.append(json_object['data'][dates]['$overall'][0]['selector_params']['property_filter_params_list'][0]['filter']['operand'])


    dataframe = pd.DataFrame({"date":dates_list,
                                "course_id":course_id,
                                "course_view":course_view, 
                                "login_success":login_success,
                                    })

    

    return dataframe
    

### Driver Code

* 29982401 - Neha
* 29997207 - Amit
* 29997221 - Swanand
* 29997295 - Varun
* 29997400 - Mortal
* 29997924 - Punit
* 29997925 - Dharmesh
* 29997932 - Raftaar
* 29997935 - Zakir
* 29997944 - Vishal

In [6]:
df_mixpannel =pd.DataFrame()
funnel_ids = [29982401,
            29997207,
            29997221,
            29997295,
            29997400,
            29997924,
            29997925,
            29997932,
            29997935,
            29997944
             ]

for ids in funnel_ids:
    response = get_mixpannel_data(ids)
    metrics = get_metrics(response)
    df_mixpannel= df_mixpannel.append(metrics)

df_mixpannel= df_mixpannel.sort_values(by=['date'], ascending=True)


/var/folders/qh/3p8ysdvd7_l6wsdl6hm_tv_m0000gn/T/ipykernel_43880/3560698731.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mixpannel= df_mixpannel.append(metrics)
/var/folders/qh/3p8ysdvd7_l6wsdl6hm_tv_m0000gn/T/ipykernel_43880/3560698731.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mixpannel= df_mixpannel.append(metrics)
/var/folders/qh/3p8ysdvd7_l6wsdl6hm_tv_m0000gn/T/ipykernel_43880/3560698731.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mixpannel= df_mixpannel.append(metrics)
/var/folders/qh/3p8ysdvd7_l6wsdl6hm_tv_m0000gn/T/ipykernel_43880/3560698731.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

## Updating numbers in google sheets

In [7]:
mixpannel_csv = f"mixpannel_data_{now}.csv"
df_mixpannel.to_csv(mixpannel_csv,index=False)

spread_sheet.values_update(
    'mixpannel_raw_data',
    params={'valueInputOption': 'USER_ENTERED'},
    body={'values': list(csv.reader(open(mixpannel_csv)))}
    )

{'spreadsheetId': '1XLTtnFKpf6zVqo-5wwsFSvljZJcooBfQ4cnKEp8lZ9Q',
 'updatedRange': 'mixpannel_raw_data!A1:D211',
 'updatedRows': 211,
 'updatedColumns': 4,
 'updatedCells': 844}